## Prediction using Tensorflow

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img

In [2]:
tf.__version__

'2.13.0'

In [ ]:
import numpy as np
from PIL import Image
import requests
from io import BytesIO
url = "https://t4.ftcdn.net/jpg/00/97/58/97/360_F_97589769_t45CqXyzjz0KXwoBZT9PRaWGHRk5hQqQ.jpg"

In [2]:
model = load_model("artifacts/cats_and_dogs.h5")

In [3]:
model

In [17]:
def load_image(path, from_url=True, process=True):
    """
    Custom preprocessing function. 
    """
    if from_url:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
    else:
        img = Image.open(path)
    if process:
        img = img.resize((224,224), Image.Resampling.LANCZOS)
        img = np.array(img, dtype=np.float32)
        img = img * (1./255)
    return np.asarray([img])

In [29]:
img = load_image(url)
img.shape

(224, 224, 3)

In [33]:
preds = model.predict(img).tolist()

1/1 [==============================] - 0s 329ms/step


In [19]:
def postprocess(preds):
    classes = ['cat', 'dog']
    return dict(zip(classes, preds))

In [34]:
postprocess(preds)

{'cat': [1.0, 6.151428758732758e-12]}

## Converting to TF-Lite

In [36]:
import tensorflow.lite as tflite 
# import tflite_runtime.interpreter as tflite

In [38]:
converter = tflite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with tf.io.gfile.GFile('artifacts/cats_and_dogs.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\dell\AppData\Local\Temp\tmp1k_qaazw\assets


INFO:tensorflow:Assets written to: C:\Users\dell\AppData\Local\Temp\tmp1k_qaazw\assets


## Inference Using TF-Lite

You will need to restart the kernel now, to be able to import tflite. You can't seem to have two interpreters (tflite and tensorflow) in the same session.

In [1]:
import tflite_runtime.interpreter as tflite

In [2]:
interpreter = tflite.Interpreter(model_path='cats_vs_dogs.tflite')
interpreter.allocate_tensors()

In [5]:
input_details = interpreter.get_input_details()
input_index = input_details[0]['index']

In [6]:
output_details = interpreter.get_output_details()
output_index = output_details[0]['index']

In [18]:
img = load_image(url)
interpreter.set_tensor(input_index, img)
interpreter.invoke()

In [20]:
preds = interpreter.get_tensor(output_index)
postprocess(preds)

{'cat': array([1.000000e+00, 6.131948e-12], dtype=float32)}

In [1]:
import requests

url = "http://localhost:8080/2015-03-31/functions/function/invocations"
# url = "https://<AWS-API-Gateway-LINK>/test/predict"

data = {"url": "https://upload.wikimedia.org/wikipedia/commons/thumb/0/02/Sea_Otter_%28Enhydra_lutris%29_%2825169790524%29_crop.jpg/800px-Sea_Otter_%28Enhydra_lutris%29_%2825169790524%29_crop.jpg"}

results = requests.post(url, json=data).json()

print(results)

ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /2015-03-31/functions/function/invocations (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000024CB78289A0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))